<a href="https://colab.research.google.com/github/enot9910/DL/blob/main/main_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import scipy
from pathlib import Path
import seaborn as sns
import lightgbm as lgb
import os
import sys

import zipfile
from google.colab import drive

def read_data(path):
    data_path = Path(path)
    data = pd.read_csv(data_path, sep = '\t', decimal=',', index_col = 0, na_values =["#ERROR!","#NUM!"])
    return data

drive.mount('/content/drive/')
train_data_subjects = read_data('drive/My Drive/Colab Notebooks/subjects_info_5.tsv')
train_data_summtest = read_data('drive/My Drive/Colab Notebooks/tests_summary_5.tsv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
train_data_summtest

,T1back_MR_mean,T1back_MR_std,T1back_MR_min,T1back_MR_25%,T1back_MR_50%,T1back_MR_75%,T1back_MR_max,T1back_SMR_mean,T1back_SMR_std,T1back_SMR_min,...,T258_t+_tau,T258_t-_diff_mean,T258_t-_diff_std,T258_t-_diff_min,T258_t-_diff_25%,T258_t-_diff_50%,T258_t-_diff_75%,T258_t-_diff_max,T258_t-_corr,T258_t-_tau
11549,116.6470588235294,12.39425863288416,95.0,111.0,112.0,127.0,144.0,570.5882352941177,342.5921516219267,420.0,...,-0.33333333333333326,-0.14696971561732636,0.4528408403193001,-0.9862349091729663,-0.4378030343015581,-0.23475679209014147,0.2063258027743265,0.6618382185866848,-0.426714365992325,-0.3589743589743589
11792,90.11764705882354,12.246848334067302,56.0,87.0,87.0,95.0,112.0,529.0588235294117,70.5128273687094,433.0,...,-0.23076923076923073,-0.11353952317917275,0.33868994107860845,-0.8844969199178645,-0.13916773306028973,-0.05450123635992006,0.06458007091118122,0.4135057471264368,-0.556686351977309,-0.5128205128205127
11888,148.0,29.87354300093264,103.0,127.5,143.0,171.0,208.0,680.2666666666667,86.5022432959868,541.0,...,-0.15384615384615383,-0.029567953923102158,0.18175390584985146,-0.32314814814814813,-0.13943435346560773,-0.07183183382370124,0.08342111366334978,0.31757866801465945,-0.36578837609195547,-0.282051282051282
11891,400.57142857142856,591.9830676471688,63.0,75.5,88.0,459.0,1584.0,725.1428571428571,589.4374234729894,329.0,...,-0.46153846153846145,-0.7520155343957912,1.9034117536121746,-5.0440771349862255,-0.4526876982684225,-0.11278274814314652,0.06779038318509747,0.8540991199629459,-0.5434890009450842,-0.5384615384615383
12388,151.33333333333334,23.423634787681152,125.0,139.0,153.0,156.0,219.0,651.8,77.66521560058733,513.0,...,-0.05128205128205127,-0.09213602799513482,0.22679002830054676,-0.46588106330501505,-0.20295273467730682,-0.06318431687571513,0.055436965853323625,0.2757046447002779,-0.811039376111453,-0.6923076923076922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51490,63.94117647058823,14.471828617331388,32.0,53.0,62.0,77.0,86.0,511.4117647058824,76.86762877142222,415.0,...,-0.17948717948717946,-0.030620497908220325,0.20771733384054422,-0.5375331174565794,-0.044739220341809785,0.027066943560406516,0.08065039700495077,0.2212287941311325,-0.5041024752206197,-0.282051282051282
51521,170.86666666666667,32.559542526328734,128.0,151.5,159.0,199.5,232.0,576.9333333333333,123.26595328027622,424.0,...,-0.3589743589743589,-0.40581113631406307,1.184879184634667,-3.8317256162915325,-0.5713509864264198,-0.18564988571114815,0.2025186900604568,0.7557591623036649,-0.51848233779405,-0.4102564102564102
51522,172.76470588235293,19.37307865236159,136.0,160.0,168.0,184.0,208.0,631.1764705882352,86.24980818392993,526.0,...,-0.2051282051282051,-0.15481058972040043,0.6076453311798684,-1.8322916666666667,-0.21573097199959926,-0.01340763251670003,0.15485052043917086,0.6158463385354142,-0.5210797823004483,-0.30769230769230765
51523,245.28571428571428,54.041904782561765,159.0,226.0,243.5,259.25,360.0,856.2142857142857,351.5981488030877,559.0,...,-0.2051282051282051,-0.10587510965225201,0.331391191456652,-0.9201592623637888,-0.24566824956731478,-0.03918310727047698,0.13238104811347795,0.26932586121641666,-0.6577538670710092,-0.5128205128205127


In [4]:
pd.options.mode.chained_assignment = None
#train_data_summtest = read_data("tests_summary.tsv")
#train_data_subjects = read_data("subjects_info.tsv")
train_data_summtest = train_data_summtest.astype(float)
train_data_subjects['age'] = train_data_subjects['age'].astype(float)
all_data = train_data_summtest.join(train_data_subjects, how='inner')
all_data = all_data.sort_values(by='age')
all_data = all_data[(all_data['age'] >= 10) & (all_data['age'] <= 80)]


import matplotlib.pyplot as plt
all_data_filter = all_data.copy()

for col in all_data.columns:
    if ('t-' in col or 't+' in col or 'MR' in col or 'SMR' in col) and 'diff' not in col and 'tau' not in col and 'corr' not in col :
        all_data_filter =  all_data_filter[all_data_filter[col] > 0]
all_data_filter.loc[all_data_filter['genders'] == 'M', 'genders'] = 0
all_data_filter.loc[all_data_filter['genders'] == 'F', 'genders'] = 1

In [4]:
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [1]:
!pip install pycaret
from pycaret.regression import *


In [10]:
exp_reg101 = setup(data = all_data_filter, target = 'age', session_id=123, normalize=True)

compare_models()


,Description,Value
0,Session id,123
1,Target,age
2,Target type,Regression
3,Original data shape,"(1021, 305)"
4,Transformed data shape,"(1021, 305)"
5,Transformed train set shape,"(714, 305)"
6,Transformed test set shape,"(307, 305)"
7,Numeric features,303
8,Categorical features,1
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,7.1693,83.5919,9.1276,0.4700,0.2403,0.2018,5.9130
et,Extra Trees Regressor,7.2840,87.6050,9.3478,0.4470,0.2449,0.2063,6.3970
lightgbm,Light Gradient Boosting Machine,7.3265,87.4333,9.3365,0.4467,0.2456,0.2060,6.4380
rf,Random Forest Regressor,7.3826,90.1250,9.4791,0.4308,0.2486,0.2094,12.4360
en,Elastic Net,7.6063,90.4075,9.4901,0.4305,0.2479,0.2151,0.2140
ada,AdaBoost Regressor,7.6557,91.7725,9.5669,0.4187,0.2520,0.2176,2.0640
lasso,Lasso Regression,7.8374,97.5133,9.8488,0.3867,0.2563,0.2231,0.2880
llar,Lasso Least Angle Regression,7.8374,97.5140,9.8488,0.3867,0.2563,0.2231,0.3880
xgboost,Extreme Gradient Boosting,7.8182,102.0649,10.0739,0.3589,0.2606,0.2180,8.1340
knn,K Neighbors Regressor,8.4180,114.3984,10.6683,0.2754,0.2769,0.2361,0.4130


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

GradientBoostingRegressor(random_state=123)

In [ ]:
lg = create_model('lightgbm')
tuned_model = tune_model(lg, optimize='MAE')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:09:24
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
ada = create_model('ada')
tuned_ada = tune_model('ada')
lightgbm = create_model('lightgbm')
tuned_lightgbm = tune_model('lightgbm')
plot_model(tuned_lightgbm, plot = 'error')
plot_model(tuned_lightgbm, plot='feature')

In [22]:
regr = [
    KNeighborsRegressor(n_neighbors=7),
    LinearRegression(),
    LGBMRegressor(learning_rate=0.001),
    SVR(gamma=2, C=1),
    DecisionTreeRegressor(max_depth=5, random_state=42),
    RandomForestRegressor(
        max_depth=5, n_estimators=10, max_features=1, random_state=42
    ),
    MLPRegressor(alpha=1, max_iter=1000, random_state=42),
    AdaBoostRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    BayesianRidge()
]


from sklearn.gaussian_process.kernels import RBF, Matern, DotProduct

param_grids = {
    KNeighborsRegressor: {
        'n_neighbors': [3, 5, 7, 8, 9, 11, 13],
        #'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan', 'chebyshev']
    },
    LGBMRegressor: {
        'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1],
        'num_leaves': [7, 9, 11, 13, 15, 31, 63],
        'max_depth': [-1, 3, 5, 7, 11, 15],
    },
    SVR: {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'gamma': ['scale', 'auto']
    },
    RandomForestRegressor: {
        'n_estimators': [10, 50, 100, 200, 500],
        'max_depth': [2, 4, 6, 8, 10, None],
        'min_samples_split': [2, 5, 10],
        'criterion': ['mse', 'friedman_mse', 'mae']
    },
    MLPRegressor: {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
        'activation': ['relu', 'tanh', 'logistic'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01]
    },
    AdaBoostRegressor: {
        'n_estimators': [20, 50, 100, 150, 200, 300, 500],
        'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5],
        'loss': ['linear', 'square', 'exponential']
    },
    GradientBoostingRegressor: {
        'n_estimators': [50, 100, 200, 500],
        'learning_rate': [0.01, 0.05, 0.1, 0.5],
        'loss': ['ls', 'lad', 'huber', 'quantile']
    },
    BayesianRidge: {
        'alpha_1': [1e-6, 1e-3, 1e-1, 1, 10],
        'alpha_2': [1e-6, 1e-3, 1e-1, 1, 10],
        'lambda_1': [1e-6, 1e-3, 1e-1, 1, 10],
        'lambda_2': [1e-6, 1e-3, 1e-1, 1, 10]
    }
}

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import pandas as pd
import numpy as np

def train_evaluate(X_train, y_train, col, clf, param_grid={}):
    best_model = None
    best_valid_mae = float('inf')  # Initialize with a very high value

    skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

    valid_mae_list = []
    train_mae_list = []
    y = pd.cut(y_train, bins=[-1, 20, 40, 55, 70, 80], labels=[0, 1, 2, 3, 4])
    #print(y, y_train, pd.cut(y_train, bins=[-1, 20, 40, 55, 70, 80], labels=[0, 1, 2, 3, 4]))
    if param_grid:
        print('GridSearchCV')
        print(clf, param_grid, skf.split(X_train[col], y))
        grid_search = GridSearchCV(clf, param_grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=skf.split(X_train[col], y)) #
        grid_search.fit(X_train[col], y_train, eval_set=[(X_valid[col], y_valid)])
        best_model = grid_search.best_estimator_
        clf = grid_search.best_estimator_

    for train_index, valid_index in skf.split(X_train[col], y): #, y_train
        X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]
        X_train_fold, X_valid_fold = X_train_fold[col], X_valid_fold[col]
        clf = clf.fit(X_train_fold, y_train_fold)

        y_pred_valid = clf.predict(X_valid_fold)
        y_pred_train = clf.predict(X_train_fold)

        valid_mae = mean_absolute_error(y_valid_fold, y_pred_valid)
        train_mae = mean_absolute_error(y_train_fold, y_pred_train)

        valid_mae_list.append(valid_mae)
        train_mae_list.append(train_mae)

        if valid_mae < best_valid_mae:
            best_model = clf
            best_valid_mae = valid_mae

    mean_valid_mae = np.mean(valid_mae_list)
    mean_train_mae = np.mean(train_mae_list)
    print(f"Модель: {clf}")
    print(f"Mean Valid MAE: {mean_valid_mae:.4f}")
    print(f"Mean Train MAE: {mean_train_mae:.4f}")
    return best_model, mean_valid_mae, mean_train_mae




from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_evaluate(X_train, y_train, col, clf, param_grid={}):
    best_model = None
    best_valid_mae = float('inf')  # Initialize with a very high value

    skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

    valid_mae_list = []
    train_mae_list = []
    y = pd.cut(y_train, bins=[-1, 20, 40, 55, 70, 80], labels=[0, 1, 2, 3, 4])
    #print(y, y_train, pd.cut(y_train, bins=[-1, 20, 40, 55, 70, 80], labels=[0, 1, 2, 3, 4]))
    if param_grid:
        print('GridSearchCV')
        print(clf, param_grid, skf.split(X_train[col], y))
        grid_search = GridSearchCV(clf, param_grid, scoring='neg_mean_absolute_error', n_jobs=-1, cv=skf.split(X_train[col], y)) #
        grid_search.fit(X_train[col], y_train)
        best_model = grid_search.best_estimator_
        clf = grid_search.best_estimator_

    for train_index, valid_index in skf.split(X_train[col], y): #, y_train
        X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]
        X_train_fold, X_valid_fold = X_train_fold[col], X_valid_fold[col]
        clf = clf.fit(X_train_fold, y_train_fold)

        y_pred_valid = clf.predict(X_valid_fold)
        y_pred_train = clf.predict(X_train_fold)

        valid_mae = mean_absolute_error(y_valid_fold, y_pred_valid)
        train_mae = mean_absolute_error(y_train_fold, y_pred_train)

        valid_mae_list.append(valid_mae)
        train_mae_list.append(train_mae)

        if valid_mae < best_valid_mae:
            best_model = clf
            best_valid_mae = valid_mae

    mean_valid_mae = np.mean(valid_mae_list)
    mean_train_mae = np.mean(train_mae_list)
    print(f"Модель: {clf}")
    print(f"Mean Valid MAE: {mean_valid_mae:.4f}")
    print(f"Mean Train MAE: {mean_train_mae:.4f}")
    return best_model, mean_valid_mae, mean_train_mae

def save_csv(file_name, results):
    try:
        df = pd.read_csv(file_name)
        df = pd.concat([df, pd.DataFrame(results)], ignore_index=True)
        df.to_csv(file_name, index=False)
    except:
        df = pd.DataFrame(results)
        df.to_csv(file_name, index=False)

def ml_regr(data, pred, columns, regr, param_grids, type_task = 1):
    scaler = StandardScaler()
    y = pd.cut(data['age'], bins=[-1, 20, 40, 55, 70, 80], labels=[0, 1, 2, 3, 4])
    X_train, X_test, y_train, y_test = train_test_split(data, data[pred], test_size=0.3, stratify=y, random_state=42)

    X_train_scaled = scaler.fit_transform(X_train)
    X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled = scaler.transform(X_test)
    X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    best_models = []
    best_valid_maes = []
    best_train_maes = []
    test_maes = []

    for i, clf in enumerate(regr):
        param_grid = param_grids.get(type(clf), {})
        print(f"Модель: {clf}")
        print(f"Параметры: {param_grid}")
        best_model, best_valid_mae, best_train_mae = train_evaluate(X_train, y_train, columns, clf, param_grid)
        best_models.append(best_model)
        best_valid_maes.append(best_valid_mae)
        best_train_maes.append(best_train_mae)
        y_pred = best_model.predict(X_test[columns])
        print(f"MAE на тестовых данных: {mean_absolute_error(y_test, y_pred):.4f}")
        test_maes.append(mean_absolute_error(y_test, y_pred))

        if (type_task == 1):
            results = {'models': best_models[-1],
            'train': best_train_maes[-1],
            'valid': best_valid_maes[-1],
            'tets': test_maes[-1]}
             file = 'res_ml_new_cols.csv'
        else:
            results = {
           'models_old_cols': best_model_old_col[-1],
           'train_old_cols': best_train_maes_old_col[-1],
           'valid_old_cols': best_valid_maes_old_col[-1],
           'test_old_cols': test_maes_old_col[-1]}
            file = 'res_ml_old_cols.csv'
        save_csv(file, results)

    best_model_idx = np.argmin(best_valid_maes)
    best_model = best_models[best_model_idx]
    print("-----------------------")
    print("Лучшая модель:")
    print(f"Модель: {best_model}")
    print(f"Mean Valid MAE: {best_valid_maes[best_model_idx]:.4f}")
    print(f"Mean Train MAE: {best_train_maes[best_model_idx]:.4f}")

    y_pred = best_model.predict(X_test[columns])
    print(f"MAE на тестовых данных: {mean_absolute_error(y_test, y_pred):.4f}")
    print("-----------------------")

    return best_models, best_train_maes, best_valid_maes, test_maes



IndentationError: unexpected indent (<ipython-input-28-fa133e98bec7>, line 146)

In [ ]:
best_models, best_train_maes, best_valid_maes, test_maes = ml_regr(all_data_filter, 'age', columns_ml_ages_new, regr, param_grids, type_task = 1)

In [27]:
best_models, best_train_maes, best_valid_maes, test_maes = ml_regr(all_data_filter, 'age', columns_ml_ages_new, regr, param_grids, type_task = 1)

Модель: KNeighborsRegressor(n_neighbors=7)
Параметры: {'n_neighbors': [3, 5, 7, 8, 9, 11, 13], 'metric': ['euclidean', 'manhattan', 'chebyshev']}
GridSearchCV
KNeighborsRegressor(n_neighbors=7) {'n_neighbors': [3, 5, 7, 8, 9, 11, 13], 'metric': ['euclidean', 'manhattan', 'chebyshev']} <generator object _BaseKFold.split at 0x78f0ecdc2c70>
Модель: KNeighborsRegressor(metric='manhattan', n_neighbors=13)
Mean Valid MAE: 7.6506
Mean Train MAE: 7.1416
MAE на тестовых данных: 7.7434
Модель: LinearRegression()
Параметры: {}
Модель: LinearRegression()
Mean Valid MAE: 11.3632
Mean Train MAE: 4.8082
MAE на тестовых данных: 19.6539
Модель: LGBMRegressor(learning_rate=0.001)
Параметры: {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1], 'num_leaves': [7, 9, 11, 13, 15, 31, 63], 'max_depth': [-1, 3, 5, 7, 11, 15]}
GridSearchCV
LGBMRegressor(learning_rate=0.001) {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1], 'num_leaves': [7, 9, 11, 13, 15, 31, 63], 'max_depth': [-1, 3, 5, 7, 11, 15]} <generator o

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1260 fits failed out of a total of 1890.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
363 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/

Модель: RandomForestRegressor(criterion='friedman_mse', max_features=1,
                      n_estimators=500, random_state=42)
Mean Valid MAE: 7.9829
Mean Train MAE: 2.9436
MAE на тестовых данных: 7.7556
Модель: MLPRegressor(alpha=1, max_iter=1000, random_state=42)
Параметры: {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)], 'activation': ['relu', 'tanh', 'logistic'], 'solver': ['adam', 'sgd'], 'alpha': [0.0001, 0.001, 0.01]}
GridSearchCV
MLPRegressor(alpha=1, max_iter=1000, random_state=42) {'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)], 'activation': ['relu', 'tanh', 'logistic'], 'solver': ['adam', 'sgd'], 'alpha': [0.0001, 0.001, 0.01]} <generator object _BaseKFold.split at 0x78f0ecdc26c0>


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perce

Модель: MLPRegressor(activation='logistic', alpha=0.01, hidden_layer_sizes=(50,),
             max_iter=1000, random_state=42, solver='sgd')
Mean Valid MAE: 8.2633
Mean Train MAE: 0.1289
MAE на тестовых данных: 7.6374
Модель: AdaBoostRegressor(random_state=42)
Параметры: {'n_estimators': [20, 50, 100, 150, 200, 300, 500], 'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5], 'loss': ['linear', 'square', 'exponential']}
GridSearchCV
AdaBoostRegressor(random_state=42) {'n_estimators': [20, 50, 100, 150, 200, 300, 500], 'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.5], 'loss': ['linear', 'square', 'exponential']} <generator object _BaseKFold.split at 0x78f0ecdc26c0>
Модель: AdaBoostRegressor(learning_rate=0.5, loss='square', n_estimators=150,
                  random_state=42)
Mean Valid MAE: 7.5426
Mean Train MAE: 5.8419
MAE на тестовых данных: 7.2441
Модель: GradientBoostingRegressor(random_state=42)
Параметры: {'n_estimators': [50, 100, 200, 500], 'learning_rate': [0.01, 0.05, 0.1, 0.5], 'loss

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
224 fits failed out of a total of 448.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
56 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py", line 420, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

Модель: GradientBoostingRegressor(learning_rate=0.01, loss='huber', n_estimators=500,
                          random_state=42)
Mean Valid MAE: 7.3514
Mean Train MAE: 4.2010
MAE на тестовых данных: 6.9986
Модель: BayesianRidge()
Параметры: {'alpha_1': [1e-06, 0.001, 0.1, 1, 10], 'alpha_2': [1e-06, 0.001, 0.1, 1, 10], 'lambda_1': [1e-06, 0.001, 0.1, 1, 10], 'lambda_2': [1e-06, 0.001, 0.1, 1, 10]}
GridSearchCV
BayesianRidge() {'alpha_1': [1e-06, 0.001, 0.1, 1, 10], 'alpha_2': [1e-06, 0.001, 0.1, 1, 10], 'lambda_1': [1e-06, 0.001, 0.1, 1, 10], 'lambda_2': [1e-06, 0.001, 0.1, 1, 10]} <generator object _BaseKFold.split at 0x78f0ecbdc120>
Модель: BayesianRidge(alpha_1=10, alpha_2=10, lambda_1=10, lambda_2=0.1)
Mean Valid MAE: 7.5738
Mean Train MAE: 6.6438
MAE на тестовых данных: 7.6893
-----------------------
Лучшая модель:
Модель: LGBMRegressor(learning_rate=0.05, max_depth=5, num_leaves=7)
Mean Valid MAE: 7.3246
Mean Train MAE: 4.2581
MAE на тестовых данных: 6.8259
-----------------------

TypeError: cannot unpack non-iterable LGBMRegressor object

In [ ]:
results = {'models': best_models,
           'train': best_train_maes,
           'valid': best_valid_maes,
           'tets': test_maes}


pd_new_cols=pd.DataFrame({'models_old_cols': best_model_old_col,
           'train_old_cols': best_train_maes_old_col,
           'valid_old_cols': best_valid_maes_old_col,
           'test_old_cols': test_maes_old_col}).to_csv('res_ml_new.csv', index=False)

from google.colab import files

files.download('res_ml_new.csv')


In [8]:
columns_ml_ages_old = ['T258_dH+_mean', 'T258_dH+_std', 'T258_dH+_min', 'T258_dH+_25%',
       'T258_dH+_50%', 'T258_dH+_75%', 'T258_dH+_max', 'T258_dH-_mean',
       'T258_dH-_std', 'T258_dH-_min', 'T258_dH-_25%', 'T258_dH-_50%',
       'T258_dH-_75%', 'T258_dH-_max', 'T258_t+_mean', 'T258_t+_std',
       'T258_t+_min', 'T258_t+_25%', 'T258_t+_50%', 'T258_t+_75%',
       'T258_t+_max', 'T258_t-_mean', 'T258_t-_std', 'T258_t-_min',
       'T258_t-_25%', 'T258_t-_50%', 'T258_t-_75%', 'T258_t-_max',
       'T258_ERR_mean', 'T258_ERR_LIM_mean', 'T274_MR_mean', 'T274_MR_std',
       'T274_MR_min', 'T274_MR_25%', 'T274_MR_50%', 'T274_MR_75%',
       'T274_MR_max', 'T274_SMR_mean', 'T274_SMR_std', 'T274_SMR_min',
       'T274_SMR_25%', 'T274_SMR_50%', 'T274_SMR_75%', 'T274_SMR_max',
       'T274_ERR_1_mean', 'T274_ERR_2_mean', 'T274_ERR_3_mean', 'T278_MR_mean',
       'T278_MR_std', 'T278_MR_min', 'T278_MR_25%', 'T278_MR_50%',
       'T278_MR_75%', 'T278_MR_max', 'T278_SMR_mean', 'T278_SMR_std',
       'T278_SMR_min', 'T278_SMR_25%', 'T278_SMR_50%', 'T278_SMR_75%',
       'T278_SMR_max', 'T278_ERR_1_mean', 'T278_ERR_2_mean',
       'T278_ERR_3_mean']

In [10]:
"""regr = [
    AdaBoostRegressor(learning_rate=0.05, random_state=42),
]


from sklearn.gaussian_process.kernels import RBF, Matern, DotProduct

param_grids = {
    AdaBoostRegressor: {
    },
}"""


In [13]:
best_models = ml_regr(all_data_filter, 'age', columns_ml_ages_new, regr, param_grids, type_task = 1)


Модель: AdaBoostRegressor(learning_rate=0.05, random_state=42)
Параметры: {}
Модель: AdaBoostRegressor(learning_rate=0.05, random_state=42)
Mean Valid MAE: 7.6366
Mean Train MAE: 6.4961
MAE на тестовых данных: 7.2671
-----------------------
Лучшая модель:
Модель: AdaBoostRegressor(learning_rate=0.05, random_state=42)
Mean Valid MAE: 7.6366
Mean Train MAE: 6.4961
MAE на тестовых данных: 7.2671
-----------------------


In [15]:
columns_ml_ages_new

Index(['T1back_MR_mean', 'T1back_MR_std', 'T1back_MR_min', 'T1back_MR_25%',
       'T1back_MR_50%', 'T1back_MR_75%', 'T1back_MR_max', 'T1back_SMR_mean',
       'T1back_SMR_std', 'T1back_SMR_min',
       ...
       'T258_t-_diff_std', 'T258_t-_diff_min', 'T258_t-_diff_25%',
       'T258_t-_diff_50%', 'T258_t-_diff_75%', 'T258_t-_diff_max',
       'T258_t-_corr', 'T258_t-_tau', 'genders', 'age_group'],
      dtype='object', length=305)